## Tp2 ift 870
- Quentin Levieux - levq2601
- Frederic Giasson -  giaf2602

In [1]:
import sklearn
import numpy as np
import scipy as sp
import pandas as pd
%matplotlib notebook
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
import numbers

In [2]:
package = pd.read_csv("Package2.csv",sep=";", error_bad_lines=False, encoding ='latin1')
display("Dimension de package.csv : ",package.shape)
product = pd.read_csv("Product2.csv",sep=";", error_bad_lines=False, encoding ='latin1')
print("Dimension de product.csv: ",product.shape)
print("Nombre d'attributs de package:", len(package.columns))
print("Nombre d'attributs de product:", len(product.columns))
print("\nColonne de package: \n",package.columns.values)
print("\nColonne de product: \n",product.columns.values)
commonAttributs = list(set(package).intersection(product))
print("\nAttributs communs au deux tables :\n",commonAttributs)

'Dimension de package.csv : '

(173887, 8)

Dimension de product.csv:  (93238, 20)
Nombre d'attributs de package: 8
Nombre d'attributs de product: 20

Colonne de package: 
 ['PRODUCTID' 'PRODUCTNDC' 'NDCPACKAGECODE' 'PACKAGEDESCRIPTION'
 'STARTMARKETINGDATE' 'ENDMARKETINGDATE' 'NDC_EXCLUDE_FLAG'
 'SAMPLE_PACKAGE']

Colonne de product: 
 ['PRODUCTID' 'PRODUCTNDC' 'PRODUCTTYPENAME' 'PROPRIETARYNAME'
 'PROPRIETARYNAMESUFFIX' 'NONPROPRIETARYNAME' 'DOSAGEFORMNAME' 'ROUTENAME'
 'STARTMARKETINGDATE' 'ENDMARKETINGDATE' 'MARKETINGCATEGORYNAME'
 'APPLICATIONNUMBER' 'LABELERNAME' 'SUBSTANCENAME'
 'ACTIVE_NUMERATOR_STRENGTH' 'ACTIVE_INGRED_UNIT' 'PHARM_CLASSES'
 'DEASCHEDULE' 'NDC_EXCLUDE_FLAG' 'LISTING_RECORD_CERTIFIED_THROUGH']

Attributs communs au deux tables :
 ['ENDMARKETINGDATE', 'PRODUCTNDC', 'PRODUCTID', 'STARTMARKETINGDATE', 'NDC_EXCLUDE_FLAG']


In [3]:
product.head(10)

,PRODUCTID,PRODUCTNDC,PRODUCTTYPENAME,PROPRIETARYNAME,PROPRIETARYNAMESUFFIX,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,STARTMARKETINGDATE,ENDMARKETINGDATE,MARKETINGCATEGORYNAME,APPLICATIONNUMBER,LABELERNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,DEASCHEDULE,NDC_EXCLUDE_FLAG,LISTING_RECORD_CERTIFIED_THROUGH
0,NaN,0002-0800,HUMAN OTC DRUG,Sterile Diluent,NaN,diluent,"INJECTION, SOLUTION",SUBCUTANEOUS,19870710,NaN,NDA,NDA018781,10,WATER,1,mL/mL,NaN,NaN,N,20201231.0
1,NaN,0002-1200,HUMAN PRESCRIPTION DRUG,Amyvid,NaN,Florbetapir F 18,"INJECTION, SOLUTION",INTRAVENOUS,20120601,NaN,NDA,NDA202008,10,FLORBETAPIR F-18,51,mCi/mL,"Radioactive Diagnostic Agent [EPC],Positron Em...",NaN,N,20211231.0
2,NaN,0002-1433,HUMAN PRESCRIPTION DRUG,Trulicity,NaN,Dulaglutide,"INJECTION, SOLUTION",SUBCUTANEOUS,20140918,NaN,BLA,BLA125469,10,DULAGLUTIDE,.75,mg/.5mL,"GLP-1 Receptor Agonist [EPC],Glucagon-Like Pep...",NaN,N,20201231.0
3,NaN,0002-1434,HUMAN PRESCRIPTION DRUG,Trulicity,NaN,Dulaglutide,"INJECTION, SOLUTION",SUBCUTANEOUS,20140918,NaN,BLA,BLA125469,10,DULAGLUTIDE,1.5,mg/.5mL,"GLP-1 Receptor Agonist [EPC],Glucagon-Like Pep...",NaN,N,20201231.0
4,NaN,0002-1436,HUMAN PRESCRIPTION DRUG,EMGALITY,NaN,galcanezumab,"INJECTION, SOLUTION",SUBCUTANEOUS,20180927,NaN,BLA,BLA761063,10,GALCANEZUMAB,120,mg/mL,NaN,NaN,N,20201231.0
5,NaN,0002-1445,HUMAN PRESCRIPTION DRUG,TALTZ,NaN,ixekizumab,"INJECTION, SOLUTION",SUBCUTANEOUS,20160322,NaN,BLA,BLA125521,10,IXEKIZUMAB,80,mg/mL,"Interleukin-17A Antagonist [EPC],Interleukin-1...",NaN,N,20201231.0
6,NaN,0002-2377,HUMAN PRESCRIPTION DRUG,EMGALITY,NaN,galcanezumab,"INJECTION, SOLUTION",SUBCUTANEOUS,20180927,NaN,BLA,BLA761063,10,GALCANEZUMAB,120,mg/mL,NaN,NaN,N,20201231.0
7,NaN,0002-3115,HUMAN PRESCRIPTION DRUG,EMGALITY,NaN,galcanezumab,"INJECTION, SOLUTION",SUBCUTANEOUS,20190604,NaN,BLA,BLA761063,10,GALCANEZUMAB,100,mg/mL,NaN,NaN,N,20201231.0
8,NaN,0002-3227,HUMAN PRESCRIPTION DRUG,Strattera,NaN,Atomoxetine hydrochloride,CAPSULE,ORAL,20021126,NaN,NDA,NDA021411,10,ATOMOXETINE HYDROCHLORIDE,10,mg/1,"Norepinephrine Reuptake Inhibitor [EPC],Norepi...",NaN,N,20211231.0
9,NaN,0002-3228,HUMAN PRESCRIPTION DRUG,Strattera,NaN,Atomoxetine hydrochloride,CAPSULE,ORAL,20021126,NaN,NDA,NDA021411,10,ATOMOXETINE HYDROCHLORIDE,25,mg/1,"Norepinephrine Reuptake Inhibitor [EPC],Norepi...",NaN,N,20211231.0


In [4]:
package.head(8)

,PRODUCTID,PRODUCTNDC,NDCPACKAGECODE,PACKAGEDESCRIPTION,STARTMARKETINGDATE,ENDMARKETINGDATE,NDC_EXCLUDE_FLAG,SAMPLE_PACKAGE
0,0002-0800_94c48759-29bb-402d-afff-9a713be11f0e,0002-0800,0002-0800-01,1 VIAL in 1 CARTON (0002-0800-01) > 10 mL in ...,19870710,NaN,N,N
1,0002-1200_35551a38-7a8d-43b8-8abd-f6cb7549e932,0002-1200,0002-1200-30,"1 VIAL, MULTI-DOSE in 1 CAN (0002-1200-30) > ...",20120601,NaN,N,N
2,0002-1200_35551a38-7a8d-43b8-8abd-f6cb7549e932,0002-1200,0002-1200-50,"1 VIAL, MULTI-DOSE in 1 CAN (0002-1200-50) > ...",20120601,NaN,N,N
3,0002-1433_42a80046-fd68-4b80-819c-a443b7816edb,0002-1433,0002-1433-61,2 SYRINGE in 1 CARTON (0002-1433-61) > .5 mL ...,20141107,NaN,N,Y
4,0002-1433_42a80046-fd68-4b80-819c-a443b7816edb,0002-1433,0002-1433-80,4 SYRINGE in 1 CARTON (0002-1433-80) > .5 mL ...,20141107,NaN,N,N
5,0002-1434_42a80046-fd68-4b80-819c-a443b7816edb,0002-1434,0002-1434-61,2 SYRINGE in 1 CARTON (0002-1434-61) > .5 mL ...,20141107,NaN,N,Y
6,0002-1434_42a80046-fd68-4b80-819c-a443b7816edb,0002-1434,0002-1434-80,4 SYRINGE in 1 CARTON (0002-1434-80) > .5 mL ...,20141107,NaN,N,N
7,0002-1436_dd7f66eb-66e1-4ad0-a593-417dbf2c74e2,0002-1436,0002-1436-11,1 SYRINGE in 1 CARTON (0002-1436-11) > 1 mL i...,20180927,NaN,N,N


# Question 1

## Types des attributs

In [5]:
print("Types des attributs pour package :\n\n",package.dtypes)
print("\nTypes des attributs pour product :\n\n",product.dtypes)

Types des attributs pour package :

 PRODUCTID              object
PRODUCTNDC             object
NDCPACKAGECODE         object
PACKAGEDESCRIPTION     object
STARTMARKETINGDATE      int64
ENDMARKETINGDATE      float64
NDC_EXCLUDE_FLAG       object
SAMPLE_PACKAGE         object
dtype: object

Types des attributs pour product :

 PRODUCTID                            object
PRODUCTNDC                           object
PRODUCTTYPENAME                      object
PROPRIETARYNAME                      object
PROPRIETARYNAMESUFFIX                object
NONPROPRIETARYNAME                   object
DOSAGEFORMNAME                       object
ROUTENAME                            object
STARTMARKETINGDATE                    int64
ENDMARKETINGDATE                    float64
MARKETINGCATEGORYNAME                object
APPLICATIONNUMBER                    object
LABELERNAME                          object
SUBSTANCENAME                        object
ACTIVE_NUMERATOR_STRENGTH            object
ACTIVE_INGR

### Valeurs Manquantes

In [6]:
manquantePackage = package[package.isnull().any(axis=1)]
#display(manquantePackage)
print(manquantePackage.shape)

manquanteProduct = product[product.isnull().any(axis=1)]
#display(manquanteProduct)
print(manquanteProduct.shape)

print("Valeurs manquante par colonne dans package:\n")
for col in package.columns:
    nb = package[[col]].isna().sum()
    print(nb)
print("Nombre total de rangees dans package :", len(package))

print("\nValeurs manquante par colonne dans product:\n")
for col in product.columns:
    nb = product[[col]].isna().sum()
    print(nb)
print("Nombre total de rangees dans product:", len(product))

(167507, 8)
(93238, 20)
Valeurs manquante par colonne dans package:

PRODUCTID    0
dtype: int64
PRODUCTNDC    1500
dtype: int64
NDCPACKAGECODE    2346
dtype: int64
PACKAGEDESCRIPTION    0
dtype: int64
STARTMARKETINGDATE    0
dtype: int64
ENDMARKETINGDATE    167431
dtype: int64
NDC_EXCLUDE_FLAG    0
dtype: int64
SAMPLE_PACKAGE    0
dtype: int64
Nombre total de rangees dans package : 173887

Valeurs manquante par colonne dans product:

PRODUCTID    1560
dtype: int64
PRODUCTNDC    0
dtype: int64
PRODUCTTYPENAME    0
dtype: int64
PROPRIETARYNAME    6
dtype: int64
PROPRIETARYNAMESUFFIX    83075
dtype: int64
NONPROPRIETARYNAME    4
dtype: int64
DOSAGEFORMNAME    0
dtype: int64
ROUTENAME    1932
dtype: int64
STARTMARKETINGDATE    0
dtype: int64
ENDMARKETINGDATE    88915
dtype: int64
MARKETINGCATEGORYNAME    0
dtype: int64
APPLICATIONNUMBER    13097
dtype: int64
LABELERNAME    0
dtype: int64
SUBSTANCENAME    2309
dtype: int64
ACTIVE_NUMERATOR_STRENGTH    2309
dtype: int64
ACTIVE_INGRED_UNIT  

### Incohérences dans package

#### Incohérences intra attribut

In [7]:
#Verifier que productId possède au moins 1 tiret 
productIdNotNull = package[~package['PRODUCTID'].isnull()]
arrayIndexInvalidProductIdForPackage = []

for index, row in productIdNotNull.iterrows():
    if "-" not in row['PRODUCTID']:
        arrayIndexInvalidProductIdForPackage.append(index)
print("Nombres de PRODUCTID n'ayant pas de tiret:",len(arrayIndexInvalidProductIdForPackage))

Nombres de PRODUCTID n'ayant pas de tiret: 0


In [8]:
ndcPackageCodeNotNull = package[~package['NDCPACKAGECODE'].isnull()]
arrayIndexInvalidNDCPackageCode = []

for index, row in ndcPackageCodeNotNull.iterrows():
    #Si le productndc ne contient pas de tiret
    if not row['NDCPACKAGECODE'].count("-")== 2:
        arrayIndexInvalidNDCPackageCode.append(index)

    elif not row['NDCPACKAGECODE'].replace('-','').isnumeric():
        arrayIndexInvalidNDCPackageCode.append(index)
        
print("Nombres de NDCPACKAGECODE n'ayant pas 2 tirets ou contenant des lettres:",len(arrayIndexInvalidNDCPackageCode))

Nombres de NDCPACKAGECODE n'ayant pas 2 tirets ou contenant des lettres: 154


In [9]:
#Verifier que le productNDC est composé de nombre et contient un tiret comme indiqué dans la documentation
productNDCNotNull = package[~package['PRODUCTNDC'].isnull()]
arrayIndexInvalidProductNDCForPackage = []

for index, row in productNDCNotNull.iterrows():
    #Si le productndc ne contient pas de tiret
    if not row['PRODUCTNDC'].count("-")== 1:
        arrayIndexInvalidProductNDCForPackage.append(index)

    elif not row['PRODUCTNDC'].replace('-','').isnumeric():
        arrayIndexInvalidProductNDCForPackage.append(index)
    
print("Nombres de PRODUCTNDC n'ayant pas de tiret ou contenant des lettres:",len(arrayIndexInvalidProductNDCForPackage))

Nombres de PRODUCTNDC n'ayant pas de tiret ou contenant des lettres: 519


In [10]:
#Aucune valeur manquante dans package pour la colonne STARTMARKETINGDATE donc on peut utiliser directement package
dateFormatNonValide = 0
arrayIndexInvalidStartMktgDateForPackage = []
for index, row in package.iterrows():
    #Si la date n'est pas dans le format indiqué alors cela retourne NaT
    date = pd.to_datetime(row['STARTMARKETINGDATE'], format='%Y%m%d', errors='coerce')
    if pd.isnull(date):
        arrayIndexInvalidStartMktgDateForPackage.append(index)
        dateFormatNonValide +=1
        
print("Nombre de dates de STARTMARKETINGDATE invalides : ",dateFormatNonValide)

Nombre de dates de STARTMARKETINGDATE invalides :  243


In [11]:
#SAMPLE_PACKAGE peut prendre une valeur parmis Y ou N
samplePackage = package[(package['SAMPLE_PACKAGE'] != 'Y') & (package['SAMPLE_PACKAGE'] != 'N')]

print("Nombre de ligne avec une valeur autre que Y ou N dans la colonne sample_Package : ",len(samplePackage))

#NDC_EXCLUDE_FLAG peut prendre une valeur parmis Y, N, E ou I
ndc_exclude_flag = package[(package['NDC_EXCLUDE_FLAG'] != 'Y') & (package['NDC_EXCLUDE_FLAG'] != 'N') & (package['NDC_EXCLUDE_FLAG'] != 'E') & (package['NDC_EXCLUDE_FLAG'] != 'I')]
print("Nombre de rangee avec une valeur autre que Y, N, E ou I dans la colonne ndc_exclude_flag : ",len(ndc_exclude_flag))

Nombre de ligne avec une valeur autre que Y ou N dans la colonne sample_Package :  0
Nombre de rangee avec une valeur autre que Y, N, E ou I dans la colonne ndc_exclude_flag :  0


In [12]:
endMarketingDateNotnull = package[(~package['ENDMARKETINGDATE'].isnull())]

dateFormatNonValide = 0
for index in range(0,len(endMarketingDateNotnull)):
    #Si la date n'est pas dans le format indiqué alors cela retourne NaT
    date = pd.to_datetime(endMarketingDateNotnull.iloc[index]['ENDMARKETINGDATE'], format='%Y%m%d', errors='coerce')
    if pd.isnull(date):
        dateFormatNonValide +=1
        
print("Nombre de dates de ENDMARKETINGDATE invalides : ",dateFormatNonValide)

Nombre de dates de ENDMARKETINGDATE invalides :  0


#### Incohérences inter attributs

In [13]:
#Dans le cas ou le PRODUCTNDC et le NDCPackageCode existent
productNDCAndNDCPackageCodeNotNull = package[(~package['PRODUCTNDC'].isnull()) & (~package['NDCPACKAGECODE'].isnull())]

arrayIndexIncoherentProductNDCAndNDCPackageCode = []
compteurIncoherence = 0
for index, row in productNDCAndNDCPackageCodeNotNull.iterrows() :
    productndc = row['PRODUCTNDC']
    ndcpkgcode = row['NDCPACKAGECODE']
    if productndc != ndcpkgcode[:len(productndc)]:
        arrayIndexIncoherentProductNDCAndNDCPackageCode.append(index)
        compteurIncoherence += 1
        
print("Nombre de PRODUCTNDC contenu dans NDCPACKAGECODE différent du PRODUCTNDC : ", compteurIncoherence)

Nombre de PRODUCTNDC contenu dans NDCPACKAGECODE différent du PRODUCTNDC :  674


In [14]:
#Verifier que PRODUCTNDC apparait au debut de PRODUCTID
#Dans le cas ou le PRODUCTNDC et le PRODUCTID existent
productNDCAndProductIdNotNull = package[(~package['PRODUCTNDC'].isnull()) & (~package['PRODUCTID'].isnull())]

arrayIndexIncoherentProductNDCAndProductIdForPackage = []
compteurIncoherence = 0
for index, row in productNDCAndProductIdNotNull.iterrows() :
    productndc = row['PRODUCTNDC']
    productid = row['PRODUCTID']
    if productndc != productid[:len(productndc)]:
        arrayIndexIncoherentProductNDCAndProductIdForPackage.append(index)
        compteurIncoherence += 1
        
print("Nombre de PRODUCTNDC contenu dans PRODUCTID différent du PRODUCTNDC : ", compteurIncoherence)

Nombre de PRODUCTNDC contenu dans PRODUCTID différent du PRODUCTNDC :  519


In [15]:
#Verifier que la date start soit avant ou égale à la date end (cependant date start peut être non valide)
datesIncoherentes = 0
arrayIndexDatesIncoherentesForPackage = []

for index, row in endMarketingDateNotnull.iterrows():
    dateEnd = pd.to_datetime(row['ENDMARKETINGDATE'])
    dateStart = pd.to_datetime(row['STARTMARKETINGDATE'])
    
    if dateEnd < dateStart :
        arrayIndexDatesIncoherentesForPackage.append(index)
        datesIncoherentes +=1
print("Nombre de dates incohérentes tel que start soit apres end, ou soit la date start est invalide : ",datesIncoherentes)

Nombre de dates incohérentes tel que start soit apres end, ou soit la date start est invalide :  18


En ce qui attrait au colonnes <b>PRODUCTID PRODUCTNDC NDCPACKAGECODE PACKAGEDESCRIPTION</b> de la table PACKAGE, il est difficile de d'avoir un critère qui permet de savoir si il y a une incoherence dans les données de la colonne. Pour ce qui est des autres colonnes, force est de constater qu'il n'y a pas d'incoherence flagrante dans les données intra colonnes. En effet, d'autant plus que les données de ces colonnes sont du type OBJECT, ils peuvent donc prendre n'importe quelle forme.

### Incohérences dans product

#### Incohérences intra attribut

In [16]:
#Verifier que productId possède au moins 1 tiret 
productIdNotNull = product[~product['PRODUCTID'].isnull()]
arrayIndexInvalidProductIdForProduct = []

for index, row in productIdNotNull.iterrows():
    if "-" not in row['PRODUCTID']:
        arrayIndexInvalidProductIdForProduct.append(index)
print("Nombres de PRODUCTID n'ayant pas de tiret:",len(arrayIndexInvalidProductIdForProduct))

Nombres de PRODUCTID n'ayant pas de tiret: 0


In [17]:
deaScheduleNotNull = product[~product['DEASCHEDULE'].isnull()] 

arrayIndexInvalidDeaSchedule = []

for index, row in deaScheduleNotNull.iterrows():
    if row['DEASCHEDULE'] not in ['CI', 'CII', 'CIII', 'CIV','CV']:
        arrayIndexInvalidDeaSchedule.append(index)
print("Nombre de DEASCHEDULE n'ayant pas de valeur valide :",len(arrayIndexInvalidDeaSchedule))


Nombre de DEASCHEDULE n'ayant pas de valeur valide : 0


In [18]:
#Verifier que le productNDC est composé de nombre et contient un tiret comme indiqué dans la documentation
productNDCNotNull = product[~product['PRODUCTNDC'].isnull()]
arrayIndexInvalidProductNDCForProduct = []

for index, row in productNDCNotNull.iterrows():
    #Si le productndc ne contient pas de tiret
    if "-" not in row['PRODUCTNDC']:
        arrayIndexInvalidProductNDCForProduct.append(index)
        
    elif not row['PRODUCTNDC'].replace('-','').isnumeric():
        arrayIndexInvalidProductNDCForProduct.append(index)
    
print("Nombres de PRODUCTNDC n'ayant pas de tiret ou contenant des lettres:",len(arrayIndexInvalidProductNDCForProduct))

Nombres de PRODUCTNDC n'ayant pas de tiret ou contenant des lettres: 513


In [19]:
#Aucune valeur manquante dans package pour la colonne STARTMARKETINGDATE donc on peut utiliser directement product
dateFormatNonValide = 0
for index in range(0,len(product)):
    #Si la date n'est pas dans le format indiqué alors cela retourne NaT
    date = pd.to_datetime(product.iloc[index]['STARTMARKETINGDATE'], format='%Y%m%d', errors='coerce')
    if pd.isnull(date):
        dateFormatNonValide +=1
        
print("Nombre de dates de STARTMARKETINGDATE invalides : ",dateFormatNonValide)

Nombre de dates de STARTMARKETINGDATE invalides :  0


In [20]:
endMarketingDateNotnull = product[(~product['ENDMARKETINGDATE'].isnull())]

arrayIndexInvalidEndMarketingDateForProduct = []
dateFormatNonValide = 0
for index, row in endMarketingDateNotnull.iterrows():
    #Si la date n'est pas dans le format indiqué alors cela retourne NaT
    date = pd.to_datetime(row['ENDMARKETINGDATE'], format='%Y%m%d', errors='coerce')
    if pd.isnull(date):
        arrayIndexInvalidEndMarketingDateForProduct.append(index)
        dateFormatNonValide +=1
        
print("Nombre de dates de ENDMARKETINGDATE invalide : ",dateFormatNonValide)

Nombre de dates de ENDMARKETINGDATE invalide :  1


In [21]:
listrecordcertifNotNull = product[~product['LISTING_RECORD_CERTIFIED_THROUGH'].isnull()]

dateFormatNonValide = 0
for index in range(0,len(listrecordcertifNotNull)):
    #Si la date n'est pas dans le format indiqué alors cela retourne NaT
    date = pd.to_datetime(listrecordcertifNotNull.iloc[index]['LISTING_RECORD_CERTIFIED_THROUGH'], format='%Y%m%d', errors='coerce')
    if pd.isnull(date):
        dateFormatNonValide +=1
print("Nombre de dates de LISTING_RECORD_CERTIFIED_THROUGH invalides: ", dateFormatNonValide)

Nombre de dates de LISTING_RECORD_CERTIFIED_THROUGH invalides:  0


In [22]:
#NDC_EXCLUDE_FLAG peut prendre une valeur parmis Y, N, E ou I
ndc_exclude_flag = product[(product['NDC_EXCLUDE_FLAG'] != 'Y') & (product['NDC_EXCLUDE_FLAG'] != 'N') & (product['NDC_EXCLUDE_FLAG'] != 'E') & (product['NDC_EXCLUDE_FLAG'] != 'I')]
print("Nombre de ligne avec une valeur autre que Y, N, E ou I dans la colonne ndc_exclude_flag : ",len(ndc_exclude_flag))

Nombre de ligne avec une valeur autre que Y, N, E ou I dans la colonne ndc_exclude_flag :  0


#### Incohérences inter attributs

In [23]:
#Dans le cas ou le PRODUCTNDC et le PRODUCTID existent
productNDCAndProductIdNotNull = product[(~product['PRODUCTNDC'].isnull()) & (~product['PRODUCTID'].isnull())]

arrayIndexIncoherentProductNDCAndProductIdForProduct = []
compteurIncoherence = 0
for index, row in productNDCAndProductIdNotNull.iterrows() :
    productndc = row['PRODUCTNDC']
    productid = row['PRODUCTID']
    if productndc != productid[:len(productndc)]:
        arrayIndexIncoherentProductNDCAndProductIdForProduct.append(index)
        compteurIncoherence += 1
        
print("Nombre de PRODUCTNDC contenu dans PRODUCTID différent du PRODUCTNDC : ", compteurIncoherence) 

Nombre de PRODUCTNDC contenu dans PRODUCTID différent du PRODUCTNDC :  513


In [24]:
#Dans le cas ou le MARKETINGCATEGORYNAME et le APPLICATIONNUMBER existent
#On ne teste pas sur MARKETINGCATEGORYNAME car il y a aucune valeur manquante
applicationNumberNotNull = product[(~product['APPLICATIONNUMBER'].isnull())]

compteurIncoherence = 0
arrayIndexIncoherentMarketingCategoryNameAndApplicationNumber = []
for index, row in applicationNumberNotNull.iterrows() :
    marketingctgname = row['MARKETINGCATEGORYNAME']
    applicationnumber = row['APPLICATIONNUMBER']
    
    if marketingctgname != applicationnumber[:len(marketingctgname)]:
        #Condition donner sur le site que si  Marketing Category is OTC Monograph Final or OTC Monograph Not Final, alor application nb sera partXXXX
        if str(marketingctgname[:3]) == "OTC" and str(applicationnumber[:4])=="part" or str(marketingctgname[:3]) == str(applicationnumber[:3]):
            #print(marketingctgname[0:3], applicationnumber[:4])
            continue
        else:    
            arrayIndexIncoherentMarketingCategoryNameAndApplicationNumber.append(index)
            compteurIncoherence += 1
print("Nombre de MARKETINGCATEGORYNAME contenu dans APPLICATIONNUMBER différent du MARKETINGCATEGORYNAME : ", compteurIncoherence)   

Nombre de MARKETINGCATEGORYNAME contenu dans APPLICATIONNUMBER différent du MARKETINGCATEGORYNAME :  139


In [25]:
#Verifier que la date start soit avant ou égale à la date end (cependant end start peut être non valide car 1 trouvée)
datesIncoherentes = 0
for index in range(0,len(endMarketingDateNotnull)):
    dateEnd = pd.to_datetime(endMarketingDateNotnull.iloc[index]['ENDMARKETINGDATE'])
    dateStart = pd.to_datetime(endMarketingDateNotnull.iloc[index]['STARTMARKETINGDATE'])
    
    if dateEnd < dateStart :
        datesIncoherentes +=1
print("Nombre de dates incohérentes tel que start soit apres end, ou soit la date start est invalide : ",datesIncoherentes)

Nombre de dates incohérentes tel que start soit apres end, ou soit la date start est invalide :  0


In [26]:
hh = product[~product['DEASCHEDULE'].isnull()]
print("Differentes possibilites pour DEASCHEDULE :",hh['DEASCHEDULE'].unique())

jj = product[~product['ACTIVE_INGRED_UNIT'].isnull()]
print("Differentes possibilites pour ACTIVE_INGRED_UNIT :",jj['ACTIVE_INGRED_UNIT'].unique())

kk = product[~product['ACTIVE_NUMERATOR_STRENGTH'].isnull()]
erreurActiveNumerator= [isinstance(x, numbers.Number) for x in kk]
print("Differentes possibilites pour ACTIVE_NUMERATOR_STRENGTH :",kk['ACTIVE_NUMERATOR_STRENGTH'].unique())
print("Il y a ",len(erreurActiveNumerator), "identifiants de ACTIVE_NUMERATOR_STRENGTH qui ne sont pas des nombres")

ll = product[~product['ROUTENAME'].isnull()]
print("Nombres de differentes possibilites pour ROUTENAME :",len(ll['ROUTENAME'].unique()))

zz = product[~product['DOSAGEFORMNAME'].isnull()]
print("Nombres de differentes possibilites pour DOSAGEFORMNAME :",len(zz['DOSAGEFORMNAME'].unique()))

cc = product[~product['PROPRIETARYNAMESUFFIX'].isnull()]
nameSuffix = []
for date in cc['PROPRIETARYNAMESUFFIX'].unique():
    if any(char.isdigit() for char in date):
           nameSuffix.append(date)
print("Nombres de differentes possibilites pour PROPRIETARYNAMESUFFIX :",len(nameSuffix))

Differentes possibilites pour DEASCHEDULE : ['CV' 'CIV' 'CIII' 'CII']
Differentes possibilites pour ACTIVE_INGRED_UNIT : ['mL/mL' 'mCi/mL' 'mg/.5mL' ... 'g/30mL; g/30mL; g/30mL; 1/30mL' '1/25mL'
 'mg/6g']
Differentes possibilites pour ACTIVE_NUMERATOR_STRENGTH : ['1' '51' '.75' ... '33.2' '415' '20; 25; 33']
Il y a  20 identifiants de ACTIVE_NUMERATOR_STRENGTH qui ne sont pas des nombres
Nombres de differentes possibilites pour ROUTENAME : 180
Nombres de differentes possibilites pour DOSAGEFORMNAME : 134
Nombres de differentes possibilites pour PROPRIETARYNAMESUFFIX : 1455


Pour la colonne <b>DEASCHEDULE</b>, nous ne connaissons pas les valeurs des attributs il est donc difficile de savoir si il y a une incoherence.
<br><br>Pour la colonne <b>PHARM_CLASSES</b>, nous ne pouvons pas evaluer les incoherences en raison des valeurs de la colonne.
<br><br>Pour la colonne <b>ACTIVE_INGRED_UNIT</b>, il y a plusieurs valeurs, cependant en les analysant visuellement, il ne semble pas y avoir d'incoherences.
<br><br>Pour la colonne <b>ACTIVE_NUMERATOR_STRENGTH</b>, il y a des valeurs qui ne sont pas des nombres. Nous ne savons pas si ce sont des incoherences cependant, nous estimons pertinent de le souligner au cas ou cela pourrait poserait probleme plus tard.
<br><br>Pour la colonne <b>SUBSTANCENAME</b> nous ne pouvons pas savoir si les attributs qui sont dans les colonnes sont bons puisque nous n'avons pas de banque permettant de valider. Il en est de meme pour <b>MARKETINGCATEGORYNAME APPLICATIONNUMBER LABELERNAME NONPROPRIETARYNAME PRODUCTID PRODUCTNDC PRODUCTTYPENAME PROPRIETARYNAME</b>, ce sont tous de objets, nous n'avons donc pas beaucoup de details quant a la composition des attributs.
<br><br>Pour la colonne <b>PROPRIETARYNAMESUFFIX</b> nous voyons qu'il y a 1455 elements que leur attribut contient un chiffre dans le suffix. Nous ne pouvons donc pas affirmer que de contenir un chiffre est un incoherence malgre que ce sont des suffix.

#### Conclusion 
Nous avons donc analysé toutes les colonnes afin de voir si il n'y avait pas d'erreurs ou d'incohérence dans les éléments de ces colonnes. Pour les dates nous avons vérifié qu'elles comportaient un mois entre 1 et 12 et un jour entre 1 et 31. Nous ne tenons pas compte de la variance du nombre de jours entre les mois. Lorsque la colonne comportait des objets qui par le nom nous permettait de croire que ce devrait être soi seulement composé de chiffres ou seulement composé de lettres, nous avons vérifié si c'était le cas et compté les nombre d'exceptions afin de vérifier si elles étaient répandues ou pas. Force est de constaté suite à notre analyse qu'il ne semble pas y avoir d'incohérence intra-attributs.

Les incohérences qu'il faudrait qu'on a verifiées : 

**Package :**

- ProductNDC presente un tiret et est constitué de nombre
- NDCPackageCode possède deux tirets
- ProductNDC present dans ProductID
- NDCPackageCode est composé de ProductNDC + autres 
- SamplePackage E {Y, N}
- NDCExcludeFlag E {Y, N, E, I}
- Start et end marketindate ils doivent être cohérent l'un doit être avant l'autre
- Verifier format de date valide pour end et start

**Product :**

- ProductNDC presente un tiret et est constitué de nombre
- ProductNDC present dans ProductID
- MarkekingCategoryName est au début d'ApplicationNumber 
- NDCExcludeFlag E {Y, N, E, I}
- Start et end marketindate ils doivent être cohérent l'un doit être avant l'autre
- Verifier format de date valide pour en, start et listingRecord

## Question2
Comme nous l'avons vu à la question 1, il y a certaines attributs qui sont reliés, qui se complète. Ci-dessous nous allons énumérer ces relations.

### Table Package
Les éléments suivants ne sont pas directement énuméré dans la table mais sont utilisés par 1 ou plusieurs éléments : SPL documentID, labeler code(labelerName dans la table Product), product code et package code de la National Drug Code number.
   - ProductID est le produit de la concaténation entre NDCproduct et SPL DocumentID.
   - Le productNDC est la concaténation entre le labelerCode et le productCode, ils sont séparés par des tirets. 
   - Le NDCPackageCode est le produit du labeler code, product code et package, ils sont séparés par des tirets. Deux de ses trois éléments sont partagés avec le productNDC.
   - Les autres éléments ne semblent pas avoir de liens avec d'autres attributs de cette table.

### Table product

Les éléments suivants ne sont pas directement énuméré dans la table mais sont utilisés par 1 ou plusieurs éléments : SPL documentID, product code et package code de la National Drug Code number.

   - **ProductID** est le produit de la concaténation entre NDCproduct et SPL DocumentID.
   - Le **productNDC** est la concaténation entre le labelerCode et le productCode, ils sont séparés par des tirets.
   - **ProprietaryNameSuffix** est composé de **ProprietaryName** suivit d'un suffix servant à distinguer un produit par caractéristiques.
   - L'attribut **MarketingCategoryName** aura l'une des étiquettes suivantes : <i>NDA/ANDA/BLA, OTC Monograph ou Unapproved Drug</i>. **ApplicationNumber** est le produit le MarketingCategoryName en ce sens qu'un code lui sera attribué selon la catégorie Marketing assignée.
   - L'attribut **SubstanceName** énumère les ingrédients actifs du produit. **StrengthNumber, StrengthUnit** est respectivement la force des ingrédients actifs identifiés et l'unité de mesure, l'ordre d'apparition des ingrédients est le même dans les deux catégories. L'attribut**Pharm_classes** est l'identifiant de la categorie pharmaceutique des substance identifié dans SubstanceName, l'ordre d'apparition est le même que dans les autres attributs. **DIRE QU ON EN FERA RIEN**

# Question 3

In [27]:
#Correction de ProductNDC et ProductId
#Il y a des cas ou il faut transformer la date et le mois en chiffre dans le bon ordre ex) 05-juin->0006-0005, nous en avons corrige 10
correction =""
for ind in arrayIndexIncoherentProductNDCAndProductIdForPackage:
    correction=""
    if str(package.iloc[ind]["PRODUCTNDC"][3:8]).isalpha():
        mois = str(package.iloc[ind]["PRODUCTNDC"][3:8])
        jour = str(package.iloc[ind]["PRODUCTNDC"][0:2])
        if mois == "sept":
            correction+="0009-00"+jour
        elif mois == "juin":
            correction+="0006-00"+jour
        package.loc[ind, "PRODUCTNDC"] = correction
        arrayIndexIncoherentProductNDCAndProductIdForPackage.remove(ind)
        
#Les incoherences qu'il reste a corriger(Je n'y vois aucune logique)
display(package.iloc[6913][["PRODUCTID","PRODUCTNDC"]])
#for ind in arrayIndexIncoherentProductNDCAndProductIdForPackage:
    # display(package.iloc[ind][["PRODUCTID","PRODUCTNDC"]])

PRODUCTID     0220-2997_8590fcc9-c0c3-f43b-e053-2991aa0a692e
PRODUCTNDC                                         0220-2997
Name: 6913, dtype: object

In [28]:
#Toute les erreurs sont la meme, l'annee est ecrit 2997 au lieu de 1997, nous corrigeons 243 erreurs
for date in arrayIndexInvalidStartMktgDateForPackage:
    nDate = "1"+str(package.iloc[date]["STARTMARKETINGDATE"])[1:]
    package.loc[date,"STARTMARKETINGDATE"]=nDate
    arrayIndexInvalidStartMktgDateForPackage.remove(date)

In [29]:
#Correction de l'erreur endMarketingDate
dd = product.iloc[arrayIndexInvalidEndMarketingDateForProduct[0]]['ENDMARKETINGDATE']
str(dd)[:]
correction = "2"+str(dd)[1:]
product.loc[arrayIndexInvalidEndMarketingDateForProduct[0],'ENDMARKETINGDATE']= correction

In [30]:
#Nous allons supprimer les lignes incohérentes dans la table package qui ne peuvent pas être corrigées 
arrayIndexToRemoveInPackage = arrayIndexDatesIncoherentesForPackage + arrayIndexIncoherentProductNDCAndNDCPackageCode + arrayIndexIncoherentProductNDCAndProductIdForPackage + arrayIndexInvalidNDCPackageCode + arrayIndexInvalidProductIdForPackage + arrayIndexInvalidStartMktgDateForPackage

arrayIndexToRemoveInPackage = list(set(arrayIndexToRemoveInPackage))

package = package.drop(package.index[arrayIndexToRemoveInPackage])

In [31]:
#Nous allons supprimer les lignes incohérentes dans la table product qui ne peuvent pas être corrigées 
arrayIndexToRemoveInProduct = arrayIndexIncoherentProductNDCAndProductIdForProduct + arrayIndexInvalidEndMarketingDateForProduct + arrayIndexInvalidProductIdForProduct + arrayIndexInvalidProductNDCForProduct

arrayIndexToRemoveInProductUnique = list(set(arrayIndexToRemoveInProduct))

product = product.drop(product.index[arrayIndexToRemoveInProductUnique])

# Question 4

Nous allons chercher à compléter les données manquantes dans les deux tables. Tout d'abord nous allons le faire dans la table package. **Nous n'esseyerons pas de spéculer sur les règles d'inférences entre les attributs, et nous préférons supprimer les lignes contenant des données manquantes.**

**Table package**

- Correction du productNDC à partir du productId car nous avons aucune données manquantes pour l'attribut productId.
- Pour les valeurs manquantes du NDCPackageCode nous n'avons pas accès au *package code segments of the national drug code number*
- Pour les endMarketingDate nous ne pourrons corriger parce que nous ne pouvons pas faire d'inférence quant à la endMarketingDate à partir des données disponibles.

**Table product**

- Correction impossible pour le productId puisque nous n'avons pas accès au *SPL document ID* 
- Correction des valeurs manquantes de proprietaryName si nous connaissons proprietaryNameSuffix
- Quant aux autres attributs nous ne pouvons pas corriger les valeurs manquantes puisque nous ne pouvons faire d'inférence à partir des données disponibles.

In [32]:
#Correction de PRODUCTNDC de la table package a partir de PRODUCTID
listeProdNDCNull = package[package['PRODUCTNDC'].isnull()].index.tolist()

for index in range(0, len(listeProdNDCNull)):
    prodId = package['PRODUCTID'][listeProdNDCNull[index]]
    prodNDC = prodId.split('_')[0]
    package.loc[listeProdNDCNull[index],'PRODUCTNDC'] = prodNDC


In [33]:
#Enlever les rangees ou NDCPACKAGECODE est null de package
listeNDCPckgCodeNull = package[package['NDCPACKAGECODE'].isnull()].index.tolist()
package=package.drop(package.index[listeNDCPckgCodeNull])

Pour ce qui est des valeurs de **ENDMARKETINGDATE** qui sont null, elles sont trop nombreuses et nous ne pouvons pas les corriger sans spéculer et avons donc décidé de les laisser tel quel.

In [34]:
#Correction des productId null de la table product
listeProdIdNull = product[product['PRODUCTID'].isnull()].index.tolist()

product = product.drop(product.index[listeProdIdNull])

In [35]:
listeProprietaryNameNull = product[product['PROPRIETARYNAME'].isnull()].index.tolist()

for index in range(0, len(listeProprietaryNameNull)):
    #propName = product.iloc[index]['PROPRIETARYNAME']
    propNameSuf = product.iloc[index]['PROPRIETARYNAMESUFFIX']
    print(propNameSuf)
    

nan
nan
nan
nan
nan
nan


Puisque les 6 valeurs de **PROPRIETARYNAMESUFFIX** correspondant aux objets ayant **PROPRIETARYNAME** null sont nan, il est impossible d'en déduire la valeur de **PROPRIETARYNAME**.

# Question 5

In [36]:
#package.drop_duplicates()
print("Nombres de lignes dupliquées de package:", package.duplicated().sum())
print("Nombres de lignes dupliquées de product:",product.duplicated().sum())

Nombres de lignes dupliquées de package: 0
Nombres de lignes dupliquées de product: 0


Comme il n'y a pas de lignes identiques, nous n'en avons pas enlevé encore. Cependant nous émettons l'hypothèse que lorsque nous éliminerons une colonne ou quand nous allons fusionner, nous aurons des lignes dupliquées.

# Question 6

Nous allons fusionner nos deux tables en fonction de l'attribut producId. Nous suivons les informations de la documentation : *It is included to help prevent duplicate rows from appearing when joining the product and package files together.*

Cependant nous allons faire la fusion sur **PRODUCTNDC** aussi afin de confirmer que **PRODUCTID** est la meilleure solution.

In [37]:
package['PRODUCTID'].describe()

count                                             170736
unique                                             91225
top       73069-037_9eff0917-36c9-4fee-8799-233396692ca3
freq                                                  72
Name: PRODUCTID, dtype: object

In [38]:
print(package['PRODUCTID'].duplicated().sum())

79511


In [39]:
package['PRODUCTNDC'].describe()

count        170736
unique        89991
top       73069-277
freq             72
Name: PRODUCTNDC, dtype: object

In [40]:
product['PRODUCTID'].describe()

count                                              91138
unique                                             91138
top       63824-267_f180df36-562c-4dbc-9de1-b3c45abcf0ee
freq                                                   1
Name: PRODUCTID, dtype: object

In [41]:
product['PRODUCTNDC'].describe()

count         91164
unique        89887
top       54365-400
freq             23
Name: PRODUCTNDC, dtype: object

In [42]:
packageSansProductId = package.copy()
packageSansProductId.drop(columns=['PRODUCTID'])

,PRODUCTNDC,NDCPACKAGECODE,PACKAGEDESCRIPTION,STARTMARKETINGDATE,ENDMARKETINGDATE,NDC_EXCLUDE_FLAG,SAMPLE_PACKAGE
0,0002-0800,0002-0800-01,1 VIAL in 1 CARTON (0002-0800-01) > 10 mL in ...,19870710,NaN,N,N
1,0002-1200,0002-1200-30,"1 VIAL, MULTI-DOSE in 1 CAN (0002-1200-30) > ...",20120601,NaN,N,N
2,0002-1200,0002-1200-50,"1 VIAL, MULTI-DOSE in 1 CAN (0002-1200-50) > ...",20120601,NaN,N,N
3,0002-1433,0002-1433-61,2 SYRINGE in 1 CARTON (0002-1433-61) > .5 mL ...,20141107,NaN,N,Y
4,0002-1433,0002-1433-80,4 SYRINGE in 1 CARTON (0002-1433-80) > .5 mL ...,20141107,NaN,N,N
...,...,...,...,...,...,...,...
173882,99207-525,99207-525-10,1 TUBE in 1 CARTON (99207-525-10) > 120 g in ...,20060313,NaN,N,N
173883,99207-525,99207-525-30,1 TUBE in 1 CARTON (99207-525-30) > 30 g in 1...,20060313,NaN,N,N
173884,99207-525,99207-525-60,1 TUBE in 1 CARTON (99207-525-60) > 60 g in 1...,20060313,NaN,N,N
173885,99207-850,99207-850-02,12 TUBE in 1 CARTON (99207-850-02) > 2 g in 1...,20131114,NaN,N,Y


In [43]:
packageSansProductId.head()
packageSansProductId.duplicated().sum()

0

In [44]:
productSansProductId = product.copy()
productSansProductId.drop(columns=['PRODUCTID'])

,PRODUCTNDC,PRODUCTTYPENAME,PROPRIETARYNAME,PROPRIETARYNAMESUFFIX,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,STARTMARKETINGDATE,ENDMARKETINGDATE,MARKETINGCATEGORYNAME,APPLICATIONNUMBER,LABELERNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,DEASCHEDULE,NDC_EXCLUDE_FLAG,LISTING_RECORD_CERTIFIED_THROUGH
20,0002-3251,HUMAN PRESCRIPTION DRUG,Strattera,NaN,Atomoxetine hydrochloride,CAPSULE,ORAL,20050214,NaN,NDA,NDA021411,10,ATOMOXETINE HYDROCHLORIDE,100,mg/1,"Norepinephrine Reuptake Inhibitor [EPC],Norepi...",NaN,N,20211231.0
21,0002-3270,HUMAN PRESCRIPTION DRUG,Cymbalta,NaN,Duloxetine hydrochloride,"CAPSULE, DELAYED RELEASE",ORAL,20100115,NaN,NDA,NDA021427,10,DULOXETINE HYDROCHLORIDE,60,mg/1,"Norepinephrine Uptake Inhibitors [MoA],Seroton...",NaN,N,20201231.0
22,0002-4112,HUMAN PRESCRIPTION DRUG,Zyprexa,NaN,Olanzapine,TABLET,ORAL,19970623,NaN,NDA,NDA020592,10,OLANZAPINE,2.5,mg/1,Atypical Antipsychotic [EPC],NaN,N,20201231.0
23,0002-4115,HUMAN PRESCRIPTION DRUG,Zyprexa,NaN,Olanzapine,TABLET,ORAL,19961001,NaN,NDA,NDA020592,10,OLANZAPINE,5,mg/1,Atypical Antipsychotic [EPC],NaN,N,20201231.0
24,0002-4116,HUMAN PRESCRIPTION DRUG,Zyprexa,NaN,Olanzapine,TABLET,ORAL,19961001,NaN,NDA,NDA020592,10,OLANZAPINE,7.5,mg/1,Atypical Antipsychotic [EPC],NaN,N,20201231.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93233,99207-465,HUMAN PRESCRIPTION DRUG,Solodyn,NaN,minocycline hydrochloride,"TABLET, FILM COATED, EXTENDED RELEASE",ORAL,20100927,NaN,NDA,NDA050808,Valeant Pharmaceuticals North America LLC,MINOCYCLINE HYDROCHLORIDE,55,mg/1,"Tetracycline-class Drug [EPC],Tetracyclines [CS]",NaN,N,20201231.0
93234,99207-466,HUMAN PRESCRIPTION DRUG,Solodyn,NaN,minocycline hydrochloride,"TABLET, FILM COATED, EXTENDED RELEASE",ORAL,20100927,NaN,NDA,NDA050808,Valeant Pharmaceuticals North America LLC,MINOCYCLINE HYDROCHLORIDE,80,mg/1,"Tetracycline-class Drug [EPC],Tetracyclines [CS]",NaN,N,20201231.0
93235,99207-467,HUMAN PRESCRIPTION DRUG,Solodyn,NaN,minocycline hydrochloride,"TABLET, FILM COATED, EXTENDED RELEASE",ORAL,20100927,NaN,NDA,NDA050808,Valeant Pharmaceuticals North America LLC,MINOCYCLINE HYDROCHLORIDE,105,mg/1,"Tetracycline-class Drug [EPC],Tetracyclines [CS]",NaN,N,20201231.0
93236,99207-525,HUMAN PRESCRIPTION DRUG,Vanos,NaN,fluocinonide,CREAM,TOPICAL,20060313,NaN,NDA,NDA021758,Valeant Pharmaceuticals North America LLC,FLUOCINONIDE,1,mg/g,"Corticosteroid [EPC],Corticosteroid Hormone Re...",NaN,N,20201231.0


In [45]:
print(productSansProductId.duplicated().sum())

0


In [46]:
tableMerged = pd.merge(package, product, left_on = "PRODUCTID", right_on="PRODUCTID", suffixes=('_PKG', '_PRD'))
print(tableMerged.shape)

(167266, 27)


In [47]:
tableMerged.head()

,PRODUCTID,PRODUCTNDC_PKG,NDCPACKAGECODE,PACKAGEDESCRIPTION,STARTMARKETINGDATE_PKG,ENDMARKETINGDATE_PKG,NDC_EXCLUDE_FLAG_PKG,SAMPLE_PACKAGE,PRODUCTNDC_PRD,PRODUCTTYPENAME,...,MARKETINGCATEGORYNAME,APPLICATIONNUMBER,LABELERNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,DEASCHEDULE,NDC_EXCLUDE_FLAG_PRD,LISTING_RECORD_CERTIFIED_THROUGH
0,0002-3251_67a53369-eead-4f2c-afe9-f3274899c47e,0002-3251,0002-3251-30,30 CAPSULE in 1 BOTTLE (0002-3251-30),20050824,NaN,N,N,0002-3251,HUMAN PRESCRIPTION DRUG,...,NDA,NDA021411,10,ATOMOXETINE HYDROCHLORIDE,100,mg/1,"Norepinephrine Reuptake Inhibitor [EPC],Norepi...",NaN,N,20211231.0
1,0002-3270_06e2a1f2-459c-45aa-9341-54e36f7726a7,0002-3270,0002-3270-04,"1000 CAPSULE, DELAYED RELEASE in 1 BOTTLE (000...",20100115,NaN,N,N,0002-3270,HUMAN PRESCRIPTION DRUG,...,NDA,NDA021427,10,DULOXETINE HYDROCHLORIDE,60,mg/1,"Norepinephrine Uptake Inhibitors [MoA],Seroton...",NaN,N,20201231.0
2,0002-3270_06e2a1f2-459c-45aa-9341-54e36f7726a7,0002-3270,0002-3270-30,"30 CAPSULE, DELAYED RELEASE in 1 BOTTLE (0002-...",20100115,NaN,N,N,0002-3270,HUMAN PRESCRIPTION DRUG,...,NDA,NDA021427,10,DULOXETINE HYDROCHLORIDE,60,mg/1,"Norepinephrine Uptake Inhibitors [MoA],Seroton...",NaN,N,20201231.0
3,0002-4112_d561034d-ea58-45fe-9d07-2e9eba98c2e4,0002-4112,0002-4112-30,30 TABLET in 1 BOTTLE (0002-4112-30),19970623,NaN,N,N,0002-4112,HUMAN PRESCRIPTION DRUG,...,NDA,NDA020592,10,OLANZAPINE,2.5,mg/1,Atypical Antipsychotic [EPC],NaN,N,20201231.0
4,0002-4115_d561034d-ea58-45fe-9d07-2e9eba98c2e4,0002-4115,0002-4115-30,30 TABLET in 1 BOTTLE (0002-4115-30),19961001,NaN,N,N,0002-4115,HUMAN PRESCRIPTION DRUG,...,NDA,NDA020592,10,OLANZAPINE,5,mg/1,Atypical Antipsychotic [EPC],NaN,N,20201231.0


In [48]:
print(tableMerged.duplicated().sum())

0


In [49]:
#Verifier que ndcExcludeFlagPrd = ndcExcludeFlagPkg
excludeFlagDifferent = tableMerged[tableMerged['NDC_EXCLUDE_FLAG_PRD'] != tableMerged['NDC_EXCLUDE_FLAG_PKG']]
print("Le nombre de NDC_EXCLUDE_FLAG_PRD différent de NDC_EXCLUDE_FLAG_PKG : ",len(excludeFlagDifferent))

Le nombre de NDC_EXCLUDE_FLAG_PRD différent de NDC_EXCLUDE_FLAG_PKG :  0


In [50]:
productIdNullInMerged = tableMerged[tableMerged['PRODUCTID'].isnull()]
print("Nombre de productId null : ", len(productIdNullInMerged))

Nombre de productId null :  0


Il est normal que nous ayons aucun **PRODUCTID** à null après avoir fait une fusion entre les deux tables, puisque nous avons choisi l'attribut **PRODUCTID** pour faire notre inner join.

In [51]:
productNDCDifferent = tableMerged[tableMerged['PRODUCTNDC_PRD'] != tableMerged['PRODUCTNDC_PKG']]

print("Nombres de PRODUCTNDC_PRD différent de PRODUCTNDC_PKG : ", len(productNDCDifferent))

Nombres de PRODUCTNDC_PRD différent de PRODUCTNDC_PKG :  0


Nous venons d'assurer que les attributs PRODUCTNDC_PRD et PRODUCTNDC_PKG sont identiques pour un même PRODUCTID. De même pour l'attribut NDC_EXCLUDE_FLAG. Nous allons donc maintenant drop une colonne de chaque pour retirer les colonnes dupliquées.

In [52]:
tableMerged = tableMerged.drop(columns=['NDC_EXCLUDE_FLAG_PRD','PRODUCTNDC_PRD'])
print(tableMerged.shape)

(167266, 25)


In [53]:
print("Valeurs manquante par colonne dans tableMerged:\n")
for col in tableMerged.columns:
    nb = tableMerged[[col]].isna().sum()
    print(round(nb/len(tableMerged),2))

Valeurs manquante par colonne dans tableMerged:

PRODUCTID    0.0
dtype: float64
PRODUCTNDC_PKG    0.0
dtype: float64
NDCPACKAGECODE    0.0
dtype: float64
PACKAGEDESCRIPTION    0.0
dtype: float64
STARTMARKETINGDATE_PKG    0.0
dtype: float64
ENDMARKETINGDATE_PKG    0.96
dtype: float64
NDC_EXCLUDE_FLAG_PKG    0.0
dtype: float64
SAMPLE_PACKAGE    0.0
dtype: float64
PRODUCTTYPENAME    0.0
dtype: float64
PROPRIETARYNAME    0.0
dtype: float64
PROPRIETARYNAMESUFFIX    0.91
dtype: float64
NONPROPRIETARYNAME    0.0
dtype: float64
DOSAGEFORMNAME    0.0
dtype: float64
ROUTENAME    0.01
dtype: float64
STARTMARKETINGDATE_PRD    0.0
dtype: float64
ENDMARKETINGDATE_PRD    0.96
dtype: float64
MARKETINGCATEGORYNAME    0.0
dtype: float64
APPLICATIONNUMBER    0.13
dtype: float64
LABELERNAME    0.0
dtype: float64
SUBSTANCENAME    0.01
dtype: float64
ACTIVE_NUMERATOR_STRENGTH    0.01
dtype: float64
ACTIVE_INGRED_UNIT    0.01
dtype: float64
PHARM_CLASSES    0.49
dtype: float64
DEASCHEDULE    0.94
dtype: flo

Ci-dessus se trouve les ratios en % des valeurs manquantes de chaque attribut. Nous allons retirer les attributs dont le ratio est supérieur à 90% de valeurs manquantes.

In [54]:
tableMerged = tableMerged.drop(columns=['ENDMARKETINGDATE_PKG','PROPRIETARYNAMESUFFIX','ENDMARKETINGDATE_PRD','DEASCHEDULE'])

In [55]:
tableMerged = tableMerged.dropna(subset=['ROUTENAME', 'APPLICATIONNUMBER','SUBSTANCENAME','ACTIVE_INGRED_UNIT','LISTING_RECORD_CERTIFIED_THROUGH', 'NDCPACKAGECODE', 'PROPRIETARYNAME','NONPROPRIETARYNAME'])

print("Valeurs manquante par colonne dans tableMerged:\n")
for col in tableMerged.columns:
    nb = tableMerged[[col]].isna().sum()
    print(round(nb/len(tableMerged),6))
print("Nombre de lignes restantes : ",len(tableMerged))

Valeurs manquante par colonne dans tableMerged:

PRODUCTID    0.0
dtype: float64
PRODUCTNDC_PKG    0.0
dtype: float64
NDCPACKAGECODE    0.0
dtype: float64
PACKAGEDESCRIPTION    0.0
dtype: float64
STARTMARKETINGDATE_PKG    0.0
dtype: float64
NDC_EXCLUDE_FLAG_PKG    0.0
dtype: float64
SAMPLE_PACKAGE    0.0
dtype: float64
PRODUCTTYPENAME    0.0
dtype: float64
PROPRIETARYNAME    0.0
dtype: float64
NONPROPRIETARYNAME    0.0
dtype: float64
DOSAGEFORMNAME    0.0
dtype: float64
ROUTENAME    0.0
dtype: float64
STARTMARKETINGDATE_PRD    0.0
dtype: float64
MARKETINGCATEGORYNAME    0.0
dtype: float64
APPLICATIONNUMBER    0.0
dtype: float64
LABELERNAME    0.0
dtype: float64
SUBSTANCENAME    0.0
dtype: float64
ACTIVE_NUMERATOR_STRENGTH    0.0
dtype: float64
ACTIVE_INGRED_UNIT    0.0
dtype: float64
PHARM_CLASSES    0.407993
dtype: float64
LISTING_RECORD_CERTIFIED_THROUGH    0.0
dtype: float64
Nombre de lignes restantes :  136941


Notre table fusionnée ne contient aucun attribut nul, et incohérent. A l'exception de l'attribut PHARM_CLASSES qui possède toujours des valeurs nulles.

# Question 7

Nous commençons par éliminer les attributs qui n'ajoutent pas d'informations (redondance d'informations).

Nous allons retirer l'attribut **LABELERNAME** qui correspond au labeler code de **PRODUCTNDC**  

In [56]:
tableMerged = tableMerged.drop(columns=['LABELERNAME'])

In [57]:
pharmClassesNotNull = tableMerged[~tableMerged['PHARM_CLASSES'].isnull()]

arrayDifferentsFieldsPharmClasses = []

for index, row in pharmClassesNotNull.iterrows():
    stringPharmClasse = row['PHARM_CLASSES']
    a = stringPharmClasse.split('[')
    for i in range (1,len(a)):
        arrayDifferentsFieldsPharmClasses.append(a[i].split(']')[0])
    
attributesPharmClasses = list(set(arrayDifferentsFieldsPharmClasses))
print(attributesPharmClasses)

['MoA', 'CS', 'EXT', 'EPC', 'PE', 'Chemical/Ingredient']


Nous observons qu'il y a 6 champs différents au sein de **PHARM_CLASSES**, nous allons donc créer 6 nouveaux attributs pour remplacer l'ancien attribut.

In [58]:
newAttributes= [[[] for ligne in range(6)] for col in range(len(tableMerged))]
indexCol = ['EPC', 'EXT', 'MoA', 'CS', 'PE', 'Chemical/Ingredient']

In [60]:
count = 0
for index, row in tableMerged.iterrows():
    str = row['PHARM_CLASSES']
    if not pd.isnull(str):
        a = str.split(']')
        
        for i in range (0,len(a)):
            substance = a[i].split('[')[0]
            if i != len(a)-1:
                classe = a[i].split('[')[1]
            
            newAttributes[count][indexCol.index(classe)].append(substance.replace(',',''))
        for j in range (0,len(indexCol)):
            newAttributes[count][j].sort()
            newAttributes[count][j] = ''.join(newAttributes[count][j]) 
        count+=1
    else :
        for j in range (0,len(indexCol)):
            newAttributes[count][j] = ''
        count+=1

Dans l'attribut PHARM_CLASS, nous avons remarqué que certaines sous-catégories pouvaient apparaîtres plusieurs fois. Nous avons donc fait la méthode précédente afin de placer les bon dans la bonne catégorie. Nous les plaçons aussi en ordre alphabétique afin de nous assurer que tous ceux qui partage les mêmes sous-attributs aient la même valeur lors de l'encodage fait plus bas. Nous avons aussi mis des string à toutes les valeurs null afin de faciliter l'encodage.

In [61]:
tableMerged[['PHARM_CLASSE_EPC', 'PHARM_CLASSE_EXT','PHARM_CLASSE_MoA','PHARM_CLASSE_CS','PHARM_CLASSE_PE','PHARM_CLASSE_Chemical/Ingredient']] = newAttributes 

Maintenant que nous venons d'obtenir une nouvelle représentation de **PHARM_CLASSES** nous pouvons retirer cet attribut

In [62]:
tableMerged = tableMerged.drop(columns=['PHARM_CLASSES'])

# Question 8

In [63]:
tableMerged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136941 entries, 0 to 167265
Data columns (total 25 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   PRODUCTID                         136941 non-null  object 
 1   PRODUCTNDC_PKG                    136941 non-null  object 
 2   NDCPACKAGECODE                    136941 non-null  object 
 3   PACKAGEDESCRIPTION                136941 non-null  object 
 4   STARTMARKETINGDATE_PKG            136941 non-null  object 
 5   NDC_EXCLUDE_FLAG_PKG              136941 non-null  object 
 6   SAMPLE_PACKAGE                    136941 non-null  object 
 7   PRODUCTTYPENAME                   136941 non-null  object 
 8   PROPRIETARYNAME                   136941 non-null  object 
 9   NONPROPRIETARYNAME                136941 non-null  object 
 10  DOSAGEFORMNAME                    136941 non-null  object 
 11  ROUTENAME                         136941 non-null  o

Nous avons décidé de retirer certains attributs et donc de réduire la dimensionnalité de notre modèle. Nous avons jugé que plusieurs attributs ne permettraient probablement pas d'améliorer la precision de notre modèle. Il est plutôt probable que ces attributs introduisent du bruit et donc réduisent la qualité des prédictions. Les attributs que nous avons éliminés servaient plutôt soit décrire à titre d'identification ou d'informations impertinentes à l'objectif de prédire la classe pharmaceutique.

In [64]:
copyTableMerged = tableMerged.copy()

In [65]:
copyTableMerged = copyTableMerged.drop(columns=['PRODUCTTYPENAME','DOSAGEFORMNAME','PACKAGEDESCRIPTION','PROPRIETARYNAME','APPLICATIONNUMBER','ACTIVE_INGRED_UNIT','SAMPLE_PACKAGE','NDC_EXCLUDE_FLAG_PKG','PRODUCTID' ,'LISTING_RECORD_CERTIFIED_THROUGH', 'STARTMARKETINGDATE_PRD', 'STARTMARKETINGDATE_PKG',"MARKETINGCATEGORYNAME"])

In [66]:
copyTableMerged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136941 entries, 0 to 167265
Data columns (total 12 columns):
 #   Column                            Non-Null Count   Dtype 
---  ------                            --------------   ----- 
 0   PRODUCTNDC_PKG                    136941 non-null  object
 1   NDCPACKAGECODE                    136941 non-null  object
 2   NONPROPRIETARYNAME                136941 non-null  object
 3   ROUTENAME                         136941 non-null  object
 4   SUBSTANCENAME                     136941 non-null  object
 5   ACTIVE_NUMERATOR_STRENGTH         136941 non-null  object
 6   PHARM_CLASSE_EPC                  136941 non-null  object
 7   PHARM_CLASSE_EXT                  136941 non-null  object
 8   PHARM_CLASSE_MoA                  136941 non-null  object
 9   PHARM_CLASSE_CS                   136941 non-null  object
 10  PHARM_CLASSE_PE                   136941 non-null  object
 11  PHARM_CLASSE_Chemical/Ingredient  136941 non-null  object
dtypes:

# Question 9

Nous allons encoder tous nos attributs avec OrdinalEncoder

In [67]:
copyTableMerged.tail()

,PRODUCTNDC_PKG,NDCPACKAGECODE,NONPROPRIETARYNAME,ROUTENAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,PHARM_CLASSE_EPC,PHARM_CLASSE_EXT,PHARM_CLASSE_MoA,PHARM_CLASSE_CS,PHARM_CLASSE_PE,PHARM_CLASSE_Chemical/Ingredient
167261,99207-525,99207-525-10,fluocinonide,TOPICAL,FLUOCINONIDE,1,Corticosteroid,,Corticosteroid Hormone Receptor Agonists,,,
167262,99207-525,99207-525-30,fluocinonide,TOPICAL,FLUOCINONIDE,1,Corticosteroid,,Corticosteroid Hormone Receptor Agonists,,,
167263,99207-525,99207-525-60,fluocinonide,TOPICAL,FLUOCINONIDE,1,Corticosteroid,,Corticosteroid Hormone Receptor Agonists,,,
167264,99207-850,99207-850-02,LULICONAZOLE,TOPICAL,LULICONAZOLE,10,Azole Antifungal,,Cytochrome P450 2C19 Inhibitors,Azoles,,
167265,99207-850,99207-850-60,LULICONAZOLE,TOPICAL,LULICONAZOLE,10,Azole Antifungal,,Cytochrome P450 2C19 Inhibitors,Azoles,,


In [68]:
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()
copyTableMerged[['ACTIVE_NUMERATOR_STRENGTH','NONPROPRIETARYNAME','PRODUCTNDC_PKG','ROUTENAME','SUBSTANCENAME','NDCPACKAGECODE','PHARM_CLASSE_EPC', 'PHARM_CLASSE_MoA','PHARM_CLASSE_PE', 'PHARM_CLASSE_CS','PHARM_CLASSE_EXT','PHARM_CLASSE_Chemical/Ingredient']] = enc.fit_transform(copyTableMerged[['ACTIVE_NUMERATOR_STRENGTH','NONPROPRIETARYNAME','PRODUCTNDC_PKG','ROUTENAME','SUBSTANCENAME','NDCPACKAGECODE','PHARM_CLASSE_EPC', 'PHARM_CLASSE_MoA','PHARM_CLASSE_PE', 'PHARM_CLASSE_CS','PHARM_CLASSE_EXT','PHARM_CLASSE_Chemical/Ingredient']])

In [69]:
display(copyTableMerged)

,PRODUCTNDC_PKG,NDCPACKAGECODE,NONPROPRIETARYNAME,ROUTENAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,PHARM_CLASSE_EPC,PHARM_CLASSE_EXT,PHARM_CLASSE_MoA,PHARM_CLASSE_CS,PHARM_CLASSE_PE,PHARM_CLASSE_Chemical/Ingredient
0,0.0,0.0,956.0,105.0,332.0,936.0,484.0,0.0,284.0,0.0,0.0,0.0
1,1.0,1.0,2650.0,105.0,1270.0,4270.0,613.0,0.0,285.0,0.0,0.0,0.0
2,1.0,2.0,2650.0,105.0,1270.0,4270.0,613.0,0.0,285.0,0.0,0.0,0.0
3,2.0,3.0,4855.0,105.0,2614.0,1905.0,112.0,0.0,0.0,0.0,0.0,0.0
4,3.0,4.0,4855.0,105.0,2614.0,3757.0,112.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
167261,69914.0,136935.0,7347.0,127.0,1498.0,681.0,209.0,0.0,121.0,0.0,0.0,0.0
167262,69914.0,136936.0,7347.0,127.0,1498.0,681.0,209.0,0.0,121.0,0.0,0.0,0.0
167263,69914.0,136937.0,7347.0,127.0,1498.0,681.0,209.0,0.0,121.0,0.0,0.0,0.0
167264,69915.0,136938.0,3766.0,127.0,2064.0,907.0,117.0,0.0,145.0,82.0,0.0,0.0


Nous avons choisi d'utiliser RandomForest comme classifieur. Il est efficace sans faire de recherche d'hyperparamètre à l'aide d'une grid search. Nous allons donc l'entrainer sur les données ayant une étiquette **PHARM_CLASS_EPC**

In [70]:
copyTableMergedTraining = copyTableMerged[copyTableMerged['PHARM_CLASSE_EPC']!=0]

In [71]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

train, test = train_test_split(copyTableMergedTraining, test_size=0.2)

y_train = train["PHARM_CLASSE_EPC"]
x_train = train.drop(columns=["PHARM_CLASSE_EPC"])
y_test = test["PHARM_CLASSE_EPC"]
x_test = test.drop(columns=["PHARM_CLASSE_EPC"])

clf = RandomForestClassifier(max_depth=5, random_state=0)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)

In [72]:
from sklearn import metrics
# Model Accuracy
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.46070307595731325


In [73]:
copyTableMergedForPrediction = copyTableMerged[copyTableMerged['PHARM_CLASSE_EPC'] == 0]

copyTableMergedForPrediction = copyTableMergedForPrediction.drop(columns=['PHARM_CLASSE_EPC'])

In [74]:
y_pred = clf.predict(x_test)

Ci-dessus nous avons entraîné un modèle sur les données ou PHARM_CLASSE_EPC n'est pas égale à 0. Nous avons ensuite avec le modèle prédit la classe des objets ayant l'attribut PHARM_CLASSE_EPC manquant, soit égale à zéro. Pour fair la diminution de dimension finale, nous avons notamment analyser la fréquence des valeurs des colonnes ainsi que la diversité des valeurs possibles. Selon ces mesures, nous avons décider de garder ou d'éliminer certaines colonnes.
<br>Force est de constater que la prédiction de notre modèle n'est pas optimale, ce qui ne peut résulter en de résultats favorable lors de la phase de prédiction. Afin d'y remédier nous aurions pu utiliser différentes techniques par exemple la cross-validation des hyper-paramêtres pour différents modèles possibles. Toutefois nous avons estimé que de pousser des recherches aussi approfondies sortait du cadre de ce tp. Dans cette remise nous estimons avoir démontrer plusieurs techniques primordiales au pré-traitement des données. Nous avons procédé de façon à garder toute les nuances des données tout en éliminant le plus d'abstraction possible. Nous avons donc bien démontré comment nettoyer un jeu de données de façon efficace afin de pouvoir modéliser les données le plus précisément possible afin de faire des prédictions sur de nouvelles données jamais vu par le modèle.